In [1]:
import pandas as pd
import os
from elasticsearch import Elasticsearch
import elasticsearch.helpers
import numpy as np
import dask.dataframe as dd
print('Importou')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Importou


In [2]:
user = 'imunizacao_public'
pwd = 'qlto5t&7r_@+#Tlstigi'
timeout = 3000
scroll = '20m'

url = {
    'host' : 'imunizacao-es.saude.gov.br',
    'port' : '443',
    'scheme':'https',
    'use_ssl' : True,
    'http_auth' : (user, pwd),
    'path' : '_search'
}
print(url)
es = Elasticsearch([url], send_get_body_as='GET', timeout=timeout)
colunas = ["vacina_categoria_codigo", "estalecimento_noFantasia", "paciente_endereco_coPais", "vacina_lote", "document_id", "paciente_endereco_nmPais", "vacina_fabricante_referencia", "vacina_nome", "estabelecimento_valor", "@timestamp", "paciente_dataNascimento", "estabelecimento_razaoSocial", "paciente_endereco_nmMunicipio", "vacina_descricao_dose", "vacina_dataAplicacao", "paciente_nacionalidade_enumNacionalidade", "estabelecimento_uf", "vacina_grupoAtendimento_codigo", "vacina_grupoAtendimento_nome", "vacina_fabricante_nome", "vacina_codigo", "status", "paciente_endereco_coIbgeMunicipio", "id_sistema_origem", "sistema_origem", "paciente_endereco_uf", "paciente_idade", "paciente_racaCor_valor", "paciente_racaCor_codigo", "paciente_enumSexoBiologico", "data_importacao_rnds", "@version", "dt_deleted", "estabelecimento_municipio_codigo", "paciente_id", "estabelecimento_municipio_nome", "paciente_endereco_cep", "vacina_categoria_nome"]
time = None
if(os.path.exists("vacina.csv")):
    dados = dd.read_csv('vacina.csv', sep = ';',usecols=['@timestamp'],dtype=str)
    time = dados['@timestamp'].max().compute()
print(time)
if(time == None):
    df = pd.DataFrame([],columns=colunas)
    df.to_csv('vacina.csv', sep = ';', encoding='utf-8-sig', index = False)
    body={
        "query": {
            "match": {"estabelecimento_uf":"MT"}
        },
        "sort":[
            {"@timestamp":"asc"}
        ]
    }
else:
    body={
        "query": {
            "bool":{
              "must":[
                {"match": {"estabelecimento_uf":"MT"}},
                {"range" : {"@timestamp" : { "gt" : str(time)}}}
              ]
            }
        },
        "sort":[
            {"@timestamp":"asc"}
        ]
    }
print(es)
print(body)
results = elasticsearch.helpers.scan(es, query=body,scroll=scroll,size=10000,request_timeout=timeout,preserve_order=True,sort=["@timestamp"])
i = 0

for document in results:
    if(i % 100000 == 0):
        print(i)

    df = pd.DataFrame([],columns=colunas)
    df = df.append(document['_source'],ignore_index=True)
    df.to_csv('vacina.csv', sep = ';', encoding='utf-8-sig', index = False, mode='a', header=False)
    i = i + 1

{'host': 'imunizacao-es.saude.gov.br', 'port': '443', 'scheme': 'https', 'use_ssl': True, 'http_auth': ('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), 'path': '_search'}
nan
<Elasticsearch([{'host': 'imunizacao-es.saude.gov.br', 'port': '443', 'scheme': 'https', 'use_ssl': True, 'http_auth': ('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), 'path': '_search'}])>
{'query': {'bool': {'must': [{'match': {'estabelecimento_uf': 'MT'}}, {'range': {'@timestamp': {'gt': 'nan'}}}]}}, 'sort': [{'@timestamp': 'asc'}]}


ConnectionError: ConnectionError(<urllib3.connection.HTTPSConnection object at 0x7fd607a82730>: Failed to establish a new connection: [Errno -2] Name or service not known) caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x7fd607a82730>: Failed to establish a new connection: [Errno -2] Name or service not known)

In [ ]:
dados = dd.read_csv('vacina.csv', sep = ';',usecols=['@timestamp'],dtype=str)
#dd.to_datetime(dados['@timestamp'],format='%Y-%m-%dT%H:%M:%S.%fZ').max().compute()
dados['@timestamp'].max().compute()

In [13]:
bruto = pd.read_csv('vacina.csv', sep = ';',usecols=['paciente_id','@timestamp'],dtype=str)
bruto[bruto['paciente_id']=='000201']

,@timestamp,paciente_id
1019156,MT,000201
1019300,MT,000201
1019318,MT,000201
1019331,MT,000201
1019373,MT,000201
...,...,...
2400654,MT,000201
2400656,MT,000201
2400665,MT,000201
2400666,MT,000201


In [42]:
def convert(time):
    print(time)
    date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
    return datetime.strptime(time, date_format)

In [98]:
dados = dd.read_csv('part-00000-a301a49c-59a7-47e0-b1fa-ea644f17f7dd.c000.csv', sep = ';',dtype=str)
dados['Faixa'] = 0
calculo = dados.groupby('paciente_idade').count().compute()['vacina_lote']
for idx, row in calculo.iteritems():
    print('idade;'+str(idx)+';qtd;'+str(row))

idade;100;qtd;302
idade;101;qtd;185
idade;102;qtd;139
idade;103;qtd;116
idade;104;qtd;71
idade;105;qtd;72
idade;109;qtd;26
idade;11;qtd;76
idade;12;qtd;36590
idade;13;qtd;37911
idade;14;qtd;39158
idade;15;qtd;42993
idade;16;qtd;45627
idade;17;qtd;44320
idade;18;qtd;71431
idade;19;qtd;71549
idade;20;qtd;73972
idade;21;qtd;79057
idade;22;qtd;79920
idade;23;qtd;83720
idade;24;qtd;87702
idade;25;qtd;92266
idade;26;qtd;94299
idade;27;qtd;91524
idade;28;qtd;90859
idade;29;qtd;89539
idade;30;qtd;90503
idade;31;qtd;91195
idade;32;qtd;96061
idade;33;qtd;95529
idade;34;qtd;94730
idade;35;qtd;97320
idade;36;qtd;94811
idade;37;qtd;94114
idade;38;qtd;96736
idade;39;qtd;96936
idade;40;qtd;94436
idade;41;qtd;92280
idade;42;qtd;90192
idade;43;qtd;88422
idade;44;qtd;86358
idade;45;qtd;84617
idade;46;qtd;82667
idade;47;qtd;78701
idade;48;qtd;77586
idade;49;qtd;77310
idade;50;qtd;75578
idade;51;qtd;74165
idade;52;qtd;73488
idade;53;qtd;72031
idade;54;qtd;69449
idade;55;qtd;69266
idade;56;qtd;67125
idade;